In [ ]:
#the simpsons
#american horror story
#how i met your mother
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
#전역 변수
url = 'https://www.springfieldspringfield.co.uk/episode_scripts.php?'

#입력 받은 값에서 띄어쓰기가 있는 경우 '-'로 변환
def replace_space(series_name):  
    
    series_name = series_name.replace(" ", "-") #띄어쓰기는 url주소에 쓰이기 위해서 '-'로 변환
    series_name.lower()    #소문자로 바꾸기
    return series_name



#대본 크롤링
def script_crawling(series_name):
    params = {
        'tv-show' : series_name
    }

    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content, 'html.parser')
    
    find_frame = soup.find('div', class_ = 'main-content-left')           #가장 바깥 프레임 설정
    find_seasons = find_frame.find_all('div', class_ = 'season-episodes') #모든 시즌 가져오기
    season_list = soup.select('.div>.season-episodes')
    print(season_list)
    seasons_list = []                                #시즌 태그와 하위 태그 값들을 모두 이곳에 저장

    for season in find_seasons:
        #시즌 이름 추출                               
        seasons_list.append(season.find('h3').text)  # 시즌 numbering   
        #에피소드 태그 추출               
        a_tags = season.find_all('a')    
        url_list = []                                #각 에피소드들의 주소가 담겨 있는 리스트
        for a in a_tags:
            url_tag = a.attrs['href']                #에피소드 별 주소 추출
            url_base = 'https://www.springfieldspringfield.co.uk/'  
            url_tag = url_base + url_tag             # 대본으로 가는 url 합치기
        
            url_list.append(url_tag)                 #에피소드 대본 url a_tag_list에 넣기
        
        seasons_list.append(url_list)                #시즌별 에피소드 url_list에 추가
    
    #에피소드별 url에 접속 후 대본 가져오기
    script_list = []
    count = 1
    for script_url in seasons_list[count]:           #seasons_list 각 인덱스에 있는 url에 접속
        script_resp = requests.get(script_url)
        script_soup = BeautifulSoup(script_resp.content, 'html.parser')
        script_list.append(script_soup.find('div', class_ = 'scrolling-script-container').text)  #대본 페이지 본문 긁어 오기
        
        #제한 설정: 시즌 2 대본까지만 긁어오기
        count += 2
        if count == 9:
            break
    
    #텍스트 파일 하나로 몰아넣기
    def save_to_txt(series_name):
        # 처음에 입력 받은 이름으로 파일 저장하기
        file_name = series_name + '_script.txt'
        print(file_name)
        with open(file_name, 'w', -1, 'utf-8') as f:   #'cp949' codec can't encode character '\xe8' in position 11518: 
                                                       #illegal multibyte sequence 에러 때문에 'utf-8' 인코딩 수정
            f.write('%s\n' % series_name)         #시즌 몇번째인지
            for scripts in script_list:      #해당 시즌 모든 에피소드 텍스트 파일에 입력
                f.write('%s\n' % scripts)
            
    save_to_txt(series_name)                 #텍스트 파일로 저장
##script crwaling end 

#미드 이미지 가져오기/et(img_url)
def get_img(series_name):

    driver_path = 'C:\\Users\\15Z970-GA5BK\\Downloads\\chromedriver.exe'
    driver = webdriver.Chrome(driver_path)
    
    driver.get('https://www.google.com')
    driver.implicitly_wait(10)
    search_input = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input')
    search_input.send_keys(series_name)

    search_btn = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[3]/center/input[1]').click()
    driver.implicitly_wait(10)
    image_btn = driver.find_element_by_link_text('이미지').click()
    
    #구글 이미지 
    # 첫번째 페이지 사진 긁어오기
    img_url = 'https://www.google.com/search?q=' + series_name + '&source=lnms&tbm=isch&sa=X&ved=0'
    resp2 = requests.get(img_url)
    soup2 = BeautifulSoup(resp2.content, 'html.parser')
    #print(soup2)
    find_img = soup2.find('table', class_ = 'images_table')
    #print(find_img)
    find_img2 = find_img.find_all('tr')
    
    # img 태그 속 src 속성 리스트에 저장하기
    img_url_list = []
    for td in find_img2:
        img_tag = td.find_all('img')
        for i in img_tag:
            img_src = i.attrs['src']
            img_url_list.append(img_src)
    

    #다음 페이지 이미지 불러오기
    img_count = 0
    img_next_url = 'https://www.google.com/search?q=' + series_name + '&biw=1036&bih=674&gbv=1&tbm=isch&ei=jWv6XKqsLo3VmAXk8ILQBg&start=' + str(img_count)
    resp3 = requests.get(img_next_url)
    soup3 = BeautifulSoup(resp3.content, 'html.parser')
    find_img = soup3.find('table', class_ = 'images_table')
    find_next_img = find_img.find_all('tr')
    
    # url에서 'start=' 값이 20씩 늘어날 때마다 다음 페이지로 넘어감
    while img_count <= 100:
        for td in find_next_img:
            img_tag = td.find_all('img')
            for i in img_tag:
                img_src = i.attrs['src']
                img_url_list.append(img_src) #기존 img_url_list에 추가로 append
            img_count += 20
        
    #image source에 접근한 후 이미지 다운로드
    count = 1
    while count <= 50:
        for i in img_url_list:
            img_url = i
            resp = requests.get(i)
            file_name = series_name  + str(count) + '.jpeg'  #번호 올리면서 이미지 저장
        
            with open(file_name, 'wb') as f:
                f.write(resp.content)     
            count += 1
            


# 함수= replace_space(), script_crawling(), #get_img()
series = replace_space(input('원하는 미드를 입력하세요: '))
#print(series) 
get_img(series)
#script_crawling(series)

# 완성

In [3]:
#the simpsons
#american horror story
#how i met your mother
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
#전역 변수
url = 'https://www.springfieldspringfield.co.uk/episode_scripts.php?'

#입력 받은 값에서 띄어쓰기가 있는 경우 '-'로 변환
def replace_space(series_name):  
    
    series_name = series_name.replace(" ", "-") #띄어쓰기는 url주소에 쓰이기 위해서 '-'로 변환
    series_name.lower()    #소문자로 바꾸기
    return series_name



#대본 크롤링
def script_crawling(series_name):
    params = {
        'tv-show' : series_name
    }

    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content, 'html.parser')
    
    find_frame = soup.find('div', class_ = 'main-content-left')           #가장 바깥 프레임 설정
    find_seasons = find_frame.find_all('div', class_ = 'season-episodes') #모든 시즌 가져오기
                                                                          #find_seasons[0] = season1

    url_list = []                                     #각 에피소드들의 주소를 담을 리스트
    url_base = 'https://www.springfieldspringfield.co.uk/'  
    for season in find_seasons:                            
        #에피소드 태그 추출               
        a_tags = season.find_all('a')    
                                      
        for a in a_tags:
            url_tag = a.attrs['href']                #에피소드 별 주소 추출            
            url_tag = url_base + url_tag             # 대본으로 가는 url 합치기        
            url_list.append(url_tag)                 #에피소드 대본 url a_tag_list에 넣기            

    print(url_list)

    #https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e01  's01e01' 만 빼오기    
    episode_list = []
    for i in url_list:    
        episode = i[-5:] 
        episode_list.append(episode)
    print(episode_list)
    
    #에피소드별 url에 접속 후 대본 가져오기
    script_list = []
    count = 0
    for script_url in url_list:           #seasons_list 각 인덱스에 있는 url에 접속
        script_resp = requests.get(script_url)
        script_soup = BeautifulSoup(script_resp.content, 'html.parser')
        script_list.append(script_soup.find('div', class_ = 'scrolling-script-container').text)  #대본 페이지 본문 긁어 오기
        
        #제한 설정: 10개 에피소드 긁어오기
        count += 1
        if count == 10:
            break
    
    #텍스트 파일 하나로 몰아넣기
    def save_to_txt(series_name):
        # 처음에 입력 받은 이름으로 파일 저장하기
        file_name = series_name + '_script.txt'
        print(file_name)
        count = 0
        with open(file_name, 'w', -1, 'utf-8') as f:   #'cp949' codec can't encode character '\xe8' in position 11518: 
                                                       #illegal multibyte sequence 에러 때문에 'utf-8' 인코딩 수정
            f.write('%s\n' % series_name)              # 미드 이름
            for scripts in script_list:                #해당 시즌 모든 에피소드 텍스트 파일에 입력
                f.write('\n-----------------------------------%s' % episode_list[count])
                f.write('\n%s' % scripts)
                count += 1
            
    save_to_txt(series_name)                 #텍스트 파일로 저장
##script crwaling end 


#미드 이미지 가져오기/et(img_url)
def get_img(series_name):

    driver_path = 'C:\\Users\\15Z970-GA5BK\\Downloads\\chromedriver.exe'
    driver = webdriver.Chrome(driver_path)
    
    driver.get('https://www.google.com')
    driver.implicitly_wait(10)
    search_input = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input')
    search_input.send_keys(series_name)

    search_btn = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[3]/center/input[1]').click()
    driver.implicitly_wait(10)
    image_btn = driver.find_element_by_link_text('이미지').click()
    
    #구글 이미지 
    #다음 페이지 이미지 불러오기
    img_count = 0   
    img_url_list = []
    while img_count < 40:
        img_next_url = 'https://www.google.com/search?q=' + series_name + \
                         '&biw=1036&bih=674&gbv=1&tbm=isch&ei=jWv6XKqsLo3VmAXk8ILQBg&start=' + str(img_count)
        # url에서 'start=' 값이 20씩 늘어날 때마다 다음 페이지로 넘어감
        resp3 = requests.get(img_next_url)
        soup3 = BeautifulSoup(resp3.content, 'html.parser')
        find_img = soup3.find('table', class_ = 'images_table')
        find_next_img = find_img.find_all('tr')
        for td in find_next_img:
            img_tag = td.find_all('img')
            for i in img_tag:
                img_src = i.attrs['src']
                img_url_list.append(img_src) #기존 img_url_list에 추가로 append
        img_count += 20
    #image source에 접근한 후 이미지 다운로드

    
    # 요기 이상함 count = 1 위치만 바꿧는데 매우 이상해짐
    count = 0
    while count <= 10:
        for i in img_url_list:
            count += 1
            img_url = i
            resp = requests.get(i)
            file_name = series_name  + str(count) + '.jpeg'  #번호 올리면서 이미지 저장
        
            with open(file_name, 'wb') as f:
                f.write(resp.content)     
            
            
            


# 함수= replace_space(), script_crawling(), #get_img()
series = replace_space(input('원하는 미드를 입력하세요: '))
#print(series) 
get_img(series)
script_crawling(series)

원하는 미드를 입력하세요: breaking bad
['https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e01', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e02', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e03', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e04', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e05', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e06', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e07', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s02e01', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s02e02', 'https://www.springfieldsp

In [ ]:
#the simpsons
#american horror story
#how i met your mother
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib import parse
#전역 변수
url = 'https://www.springfieldspringfield.co.uk/episode_scripts.php?'

#입력 받은 값에서 띄어쓰기가 있는 경우 '-'로 변환
def replace_space(series_name):  
    
    series_name = series_name.replace(" ", "-") #띄어쓰기는 url주소에 쓰이기 위해서 '-'로 변환
    series_name.lower()    #소문자로 바꾸기
    return series_name



#대본 크롤링
def script_crawling(series_name):
    params = {
        'tv-show' : series_name
    }

    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content, 'html.parser')
    
    season_list = soup.select('.season-episodes')
    print(season_list[1])
    seasons_list = []                                #시즌 태그와 하위 태그 값들을 모두 이곳에 저장
    season_dict = {}
    for season_li in season_list:
        link_url = season_li.a['href']
        episode = get_param_from_href(link_url, 'episode')
    
        season_dict[series_name] ={
            'episode' : episode,
            'link' : link_url
        }
    print(season_dict)
def get_param_from_href(href, param):
    url = parse.urlparse(href)
    url_query = parse.parse_qs(url.query)
    return url_query[param][0]

series = replace_space(input('원하는 미드를 입력하세요: '))
#print(series) 
#get_img(series)
script_crawling(series)


    



In [ ]:
#the simpsons
#american horror story
#how i met your mother
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
#전역 변수
url = 'https://www.springfieldspringfield.co.uk/episode_scripts.php?'

#입력 받은 값에서 띄어쓰기가 있는 경우 '-'로 변환
def replace_space(series_name):  
    
    series_name = series_name.replace(" ", "-") #띄어쓰기는 url주소에 쓰이기 위해서 '-'로 변환
    series_name.lower()    #소문자로 바꾸기
    return series_name



#대본 크롤링
def script_crawling(series_name):
    params = {
        'tv-show' : series_name
    }

    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content, 'html.parser')
    
    find_frame = soup.find('div', class_ = 'main-content-left')           #가장 바깥 프레임 설정
    find_seasons = find_frame.find_all('div', class_ = 'season-episodes') #모든 시즌 가져오기


    url_list = []                                     #각 에피소드들의 주소가 담겨 있는 리스트
    url_base = 'https://www.springfieldspringfield.co.uk/'  
    for season in find_seasons:                            
        #에피소드 태그 추출               
        a_tags = season.find_all('a')    
                                      
        for a in a_tags:
            url_tag = a.attrs['href']                #에피소드 별 주소 추출            
            url_tag = url_base + url_tag             # 대본으로 가는 url 합치기        
            url_list.append(url_tag)                 #에피소드 대본 url a_tag_list에 넣기            

    print(url_list)
    
    episode_list = []
    for i in url_list:    
        episode = i[-5:] #https://www.springfieldspringfield.co.uk/view_episode_scripts.php?
                            #tv-show=breaking-bad&episode=s01e01  's01e01' 만 빼오기
        episode_list.append(episode)
    print(episode_list)
    #에피소드별 url에 접속 후 대본 가져오기
    script_list = []
    count = 0
    for script_url in url_list:           #seasons_list 각 인덱스에 있는 url에 접속
        script_resp = requests.get(script_url)
        script_soup = BeautifulSoup(script_resp.content, 'html.parser')
        script_list.append(script_soup.find('div', class_ = 'scrolling-script-container').text)  #대본 페이지 본문 긁어 오기
        
        #제한 설정: 10개 에피소드 긁어오기
        count += 1
        if count == 10:
            break
    
    #텍스트 파일 하나로 몰아넣기
    def save_to_txt(series_name):
        # 처음에 입력 받은 이름으로 파일 저장하기
        file_name = series_name + '_script.txt'
        print(file_name)
        count = 0
        with open(file_name, 'w', -1, 'utf-8') as f:   #'cp949' codec can't encode character '\xe8' in position 11518: 
                                                       #illegal multibyte sequence 에러 때문에 'utf-8' 인코딩 수정
            f.write('%s\n' % series_name)         # 미드 이름
            for scripts in script_list:      #해당 시즌 모든 에피소드 텍스트 파일에 입력
                
                f.write('%s\n' % episode_list[count])
                f.write('%s\n' % scripts)
                count += 1
            
    save_to_txt(series_name)                 #텍스트 파일로 저장
##script crwaling end 

# 함수= replace_space(), script_crawling(), #get_img()
series = replace_space(input('원하는 미드를 입력하세요: '))
#print(series) 
#get_img(series)
script_crawling(series)